In [98]:
import arviz as az
import numpy as np
import pandas as pd
import polars as ps
import pymc3 as pm
from sklearn.model_selection import train_test_split
from theano import shared

In [22]:
type_map: list = [ps.Int32, ps.Int32, ps.Int32, ps.Int32, ps.Int32, ps.Int32, ps.Int32, 
                  ps.Int32, ps.Float32, ps.Float32, ps.Float32, ps.Float32, ps.Float32, 
                  ps.Float32, ps.Float32, ps.Float32, ps.Float32, ps.Float32, 
                  ps.Float32, ps.Int32, ps.Float32, ps.Float32, ps.Float32]

In [84]:
dataset: ps.DataFrame = ps.read_csv('full_dataset_train.csv', dtypes=type_map, encoding='utf8', n_threads=8, use_pyarrow=True)

In [85]:
dataset.head()

SiteId,PublishYear,PublishMonth,PublishDay,PublishHour,AQI,Pollutant,Status,SO2,SO2_AVG,CO,CO_8hr,O3,O3_8hr,PM10,PM10_AVG,NO,NO2,NOx,WindDirec,WindSpeed,PM2.5,PM2.5_AVG
i32,i32,i32,i32,i32,i32,i32,i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i32,f32,f32,f32
67,2022,6,1,1,38,1,0,0.5,0.0,0.8,1.4,1.3,1.1,13.0,14.0,25.299999,23.0,48.400002,188,1.7,4.0,7.0
44,2022,6,1,1,22,0,0,0.9,1.0,0.1,0.1,7.7,10.2,18.0,16.0,0.9,4.4,5.3,147,1.1,7.0,6.0
20,2022,6,1,1,28,0,0,0.8,0.0,0.3,0.3,8.0,11.0,21.0,17.0,0.3,14.1,14.4,136,0.2,14.0,8.0
14,2022,6,1,1,27,1,0,0.0,0.0,0.3,0.4,1.9,6.5,20.0,11.0,4.2,16.700001,20.9,133,0.6,6.0,6.0
72,2022,6,1,1,15,1,0,1.4,1.0,0.2,0.2,6.1,10.2,24.0,15.0,1.4,3.8,5.2,149,0.3,5.0,2.0


# Features

In [100]:
features: list = ['AQI', 'Status', 'PM10', 'PM10_AVG', 'PM2.5_AVG']
target_feature: str = 'PM2.5'

In [101]:
variables: np.array = np.array([dataset[feature].to_numpy() for feature in features])
variables = variables.transpose()

In [102]:
target: np.array = dataset[target_feature].to_numpy().reshape(1, -1)
target = target.transpose()

In [103]:
train_variables, test_variables, train_target, test_target = train_test_split(variables, target, 
                                                                              test_size=0.4, random_state=37710, 
                                                                              shuffle=True)

In [104]:
train_variables = shared(train_variables.transpose())
train_target = train_target.transpose()
test_variables = test_variables.transpose()
test_target = test_target.transpose()

In [105]:
# train_variables: list = [shared(train_variables[i]) 
#                          for i in range(train_variables.shape[0])]
# test_variables: list = [shared(test_variables[i]) 
#                         for i in range(test_variables.shape[0])]

# Model

In [97]:
with pm.Model() as predictor:
	intercept = pm.Normal(name='intercept', sd=10)
	coefficients = pm.Normal(name='coefficients', mu=0, sd=1, shape=5)
	error = pm.HalfCauchy(name='error', beta=5)
	model = intercept + pm.math.dot(train_variables, coefficients)
	
	prediction = pm.Normal(name='prediction', mu=model, sd=error, observed=train_target)

	trace = pm.sample(tune=2000, chains=2, cores=8)
	ppc = pm.sample_posterior_predictive(trace, samples=2000)


You can find the C code in this temporary file: /var/folders/vv/93wphtyn4w15b8hssz_f1c500000gn/T/theano_compilation_error_qzm0vifh
library atest/envs/BDA/include/python3.9/Python.h:27:5: is not found.
library atest/envs/BDA/include/python3.9/Python.h:30: is not found.
library atest/envs/BDA/bin/../include/c++/v1/string.h:95:102: is not found.
library atest/envs/BDA/bin/../include/c++/v1/string.h:98:90: is not found.
library atest/envs/BDA/include/python3.9/Python.h:34: is not found.
library atest/envs/BDA/bin/../include/c++/v1/stdlib.h:150:34: is not found.
library atest/envs/BDA/bin/../include/c++/v1/stdlib.h:151:12: is not found.
library atest/envs/BDA/bin/../include/c++/v1/stdlib.h:154:34: is not found.
library atest/envs/BDA/bin/../include/c++/v1/stdlib.h:156:12: is not found.
library atest/envs/BDA/include/python3.9/Python.h:36:10: is not found.


Exception: ('Compilation failed (return status=1): In file included from /Users/hermeschen/.theano/compiledir_macOS-14.0-x86_64-i386-64bit-i386-3.9.16-64/tmp5moyqjd4/mod.cpp:1:. /Users/hermeschen/.pyenv/versions/miniconda3-latest/envs/BDA/include/python3.9/Python.h:27:5: error: "Python.h requires that stdio.h define NULL.". #   error "Python.h requires that stdio.h define NULL.".     ^. In file included from /Users/hermeschen/.theano/compiledir_macOS-14.0-x86_64-i386-64bit-i386-3.9.16-64/tmp5moyqjd4/mod.cpp:1:. In file included from /Users/hermeschen/.pyenv/versions/miniconda3-latest/envs/BDA/include/python3.9/Python.h:30:. /Users/hermeschen/.pyenv/versions/miniconda3-latest/envs/BDA/bin/../include/c++/v1/string.h:95:102: error: unknown type name \'size_t\'. inline _LIBCPP_HIDE_FROM_ABI _LIBCPP_PREFERRED_OVERLOAD const void* memchr(const void* __s, int __c, size_t __n) {.                                                                                                      ^. /Users/herm

# Performance Analysis

In [61]:
az.plot_trace(trace, var_names=['intercept', 'coefficients', 'error'])

NameError: name 'trace' is not defined

In [ ]:
summary: pd.DataFrame = az.summary(trace, var_names=['intercept', 'coefficients', 'error'])
summary

## MSE

In [108]:
number_of_test_data: int = len(test_target)
formula_intercept: float = summary['mean'].tolist()[0]
formula_coefficients: np.array  = np.array(summary['mean'].to_list[1:1+len(features)])
mean_squared_error: float = 0
for i in range(number_of_test_data):
	mean_squared_error += (sum(np.dot(test_variables[i], formula_coefficients)) + formula_intercept - test_target[i])**2
mean_squared_error /= number_of_test_data

NameError: name 'summary' is not defined

In [111]:
print('MSE:{:.2f}'.format(mean_squared_error))

MSE:1.24
